In [2]:
'''
Transformer输入：   (seq_len, batch_size, C)
1DConv输入：        (batch_size, C, seq_len)
2DConv输入：        (batch_size, C, H, W)
Linear输入：        仅对最后一个维度从输入变成输出
'''
import torch
import torch.nn as nn
from net.mytransformer import PositionalEncoding,TransformerWithPooling
from net.myswinunet import SwinTransformerSys
# import torch.nn.functional as F
from pytictoc import TicToc
from net.utils import get_model_memory_nolog
from net.jxtnet_pureTrans import TVL1Loss
def checksize(x):
    1
    print(x.shape, torch.prod(torch.tensor(x.shape)).item())

t = TicToc()
t.tic()

#------------------------------参数设置
cudadevice = 'cuda:0'
device = torch.device(cudadevice if torch.cuda.is_available() else "cpu")
tokenlength = 22500
hiddendim = 576
input_matrix = torch.randn(6, tokenlength, hiddendim).to(device)  # batchsize length dim
out_dim = 12
#------------------------------模型初始化
num_layers = 10
pool_size = 2  # 每次减少一半的序列长度

transformer_model = TransformerWithPooling(d_model=hiddendim, nhead=4, dim_feedforward=256, num_layers=num_layers, pool_size=pool_size, activation='silu').to(device)
get_model_memory_nolog(transformer_model)
pe = PositionalEncoding(d_model=hiddendim).to(device)
conv1d1 = nn.Conv1d(576, 1, kernel_size=1, stride=1, dilation=1 ,padding=0).to(device)
get_model_memory_nolog(conv1d1)
fc1d1 = nn.Sequential(
        nn.Linear(21, 21),
        nn.SiLU(),
        nn.Linear(21, out_dim*8*45*90),
        nn.LayerNorm(out_dim*8*45*90)).to(device)
# fc1d1 = nn.Linear(351, 8*45*90)
get_model_memory_nolog(fc1d1)
swinunet = SwinTransformerSys(embed_dim=out_dim,window_size=9).to(device)
get_model_memory_nolog(swinunet)

#------------------------------数据流正文----------------------------
checksize(input_matrix)
# tic=toc(tic)
x = input_matrix.reshape(tokenlength, input_matrix.shape[0], -1)  # Transformer输入：Reshape to (seq_len, batch_size, input_channel) (L B C)
checksize(x)

#---------------Transformer Encoder----------------
print('进入Encoder')
x = pe(x)
x = transformer_model(x)  # 传入自定义的 Transformer 模型
# tic=toc(tic)

#---------------conv1d+fc bottleneck---------------
print("进入bottleneck")
x = x.reshape(x.shape[1], x.shape[2], -1)  # 1DConv输入：Reshape to (batch_size, input_channel, seq_len)
checksize(x)
x = conv1d1(x)
checksize(x)
x = fc1d1(x)
checksize(x)
# tic=toc(tic)

#-------------SwinTransformer Decoder--------------
print("进入Decoder")
x = x.reshape(x.shape[0],45*90,-1)
checksize(x)
x = swinunet(x)
checksize(x)
t.toc('准备及推理时间',restart=True)

l1loss = TVL1Loss(beta=1.0)
gt = torch.randn(6, 1, 360, 720).to(device)
loss = l1loss(x,gt).mean()
# print(loss)
loss.backward()
t.toc('反传时间',restart=True)




模型占用0.0606GB
模型占用0.0000GB
模型占用0.0348GB
SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:1000
---final upsample expand_first---
模型占用0.0004GB
torch.Size([6, 22500, 576]) 77760000
torch.Size([22500, 6, 576]) 77760000
进入Encoder
torch.Size([11250, 6, 576]) 38880000
torch.Size([5625, 6, 576]) 19440000
torch.Size([2812, 6, 576]) 9718272
torch.Size([1406, 6, 576]) 4859136
torch.Size([703, 6, 576]) 2429568
torch.Size([351, 6, 576]) 1213056
torch.Size([175, 6, 576]) 604800
torch.Size([87, 6, 576]) 300672
torch.Size([43, 6, 576]) 148608
torch.Size([21, 6, 576]) 72576
进入bottleneck
torch.Size([6, 576, 21]) 72576
torch.Size([6, 1, 21]) 126
torch.Size([6, 1, 388800]) 2332800
进入Decoder
torch.Size([6, 4050, 96]) 2332800
torch.Size([6, 16200, 48]) 4665600
torch.Size([6, 64800, 24]) 9331200
torch.Size([6, 259200, 12]) 18662400
torch.Size([6, 259200, 12]) 18662400
torch.Size([6, 1, 360, 720]) 1555200
准备及推理时间 1.245736 seconds.
反传时间 1.899117

8层encoder

In [ ]:
'''
Transformer输入：   (seq_len, batch_size, C)
1DConv输入：        (batch_size, C, seq_len)
2DConv输入：        (batch_size, C, H, W)
Linear输入：        仅对最后一个维度从输入变成输出
'''
import torch
import torch.nn as nn
from net.mytransformer import PositionalEncoding,TransformerWithPooling
from net.myswinunet import SwinTransformerSys
# import torch.nn.functional as F
from pytictoc import TicToc
from net.utils import get_model_memory_nolog
from net.jxtnet_pureTrans import TVL1Loss
def checksize(x):
    1
    print(x.shape, torch.prod(torch.tensor(x.shape)).item())

t = TicToc()
t.tic()

#------------------------------参数设置
cudadevice = 'cuda:0'
device = torch.device(cudadevice if torch.cuda.is_available() else "cpu")
tokenlength = 22500
hiddendim = 576
input_matrix = torch.randn(6, tokenlength, hiddendim).to(device)  # batchsize length dim
out_dim = 12
#------------------------------模型初始化
num_layers = 8
pool_size = 2  # 每次减少一半的序列长度

transformer_model = TransformerWithPooling(d_model=hiddendim, nhead=4, dim_feedforward=256, num_layers=num_layers, pool_size=pool_size, activation='silu').to(device)
get_model_memory_nolog(transformer_model)
pe = PositionalEncoding(d_model=hiddendim).to(device)
conv1d1 = nn.Conv1d(576, 1, kernel_size=1, stride=1, dilation=1 ,padding=0).to(device)
get_model_memory_nolog(conv1d1)
fc1d1 = nn.Sequential(
        nn.Linear(87, 87),
        nn.SiLU(),
        nn.Linear(87, out_dim*8*45*90),
        nn.LayerNorm(out_dim*8*45*90)).to(device)
# fc1d1 = nn.Linear(351, 8*45*90)
get_model_memory_nolog(fc1d1)
swinunet = SwinTransformerSys(embed_dim=out_dim,window_size=9).to(device)
get_model_memory_nolog(swinunet)

#------------------------------数据流正文----------------------------
checksize(input_matrix)
# tic=toc(tic)
x = input_matrix.reshape(tokenlength, input_matrix.shape[0], -1)  # Transformer输入：Reshape to (seq_len, batch_size, input_channel) (L B C)
checksize(x)

#---------------Transformer Encoder----------------
print('进入Encoder')
x = pe(x)
x = transformer_model(x)  # 传入自定义的 Transformer 模型
# tic=toc(tic)

#---------------conv1d+fc bottleneck---------------
print("进入bottleneck")
x = x.reshape(x.shape[1], x.shape[2], -1)  # 1DConv输入：Reshape to (batch_size, input_channel, seq_len)
checksize(x)
x = conv1d1(x)
checksize(x)
x = fc1d1(x)
checksize(x)
# tic=toc(tic)

#-------------SwinTransformer Decoder--------------
print("进入Decoder")
x = x.reshape(x.shape[0],45*90,-1)
checksize(x)
x = swinunet(x)
checksize(x)
t.toc('准备及推理时间',restart=True)

l1loss = TVL1Loss(beta=1.0)
gt = torch.randn(6, 1, 360, 720).to(device)
loss = l1loss(x,gt).mean()
# print(loss)
loss.backward()
t.toc('反传时间',restart=True)




模型占用0.0485GB
模型占用0.0000GB
模型占用0.1304GB
SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:1000
---final upsample expand_first---
模型占用0.0004GB
torch.Size([6, 22500, 576]) 77760000
torch.Size([22500, 6, 576]) 77760000
进入Encoder
torch.Size([11250, 6, 576]) 38880000
torch.Size([5625, 6, 576]) 19440000
torch.Size([2812, 6, 576]) 9718272
torch.Size([1406, 6, 576]) 4859136
torch.Size([703, 6, 576]) 2429568
torch.Size([351, 6, 576]) 1213056
torch.Size([175, 6, 576]) 604800
torch.Size([87, 6, 576]) 300672
进入bottleneck
torch.Size([6, 576, 87]) 300672
torch.Size([6, 1, 87]) 522
torch.Size([6, 1, 388800]) 2332800
进入Decoder
torch.Size([6, 4050, 96]) 2332800
torch.Size([6, 16200, 48]) 4665600
torch.Size([6, 64800, 24]) 9331200
torch.Size([6, 259200, 12]) 18662400
torch.Size([6, 259200, 12]) 18662400
torch.Size([6, 1, 360, 720]) 1555200
准备及推理时间 1.423010 seconds.
反传时间 1.887860 seconds.


4层encoder

In [ ]:
'''
Transformer输入：   (seq_len, batch_size, C)
1DConv输入：        (batch_size, C, seq_len)
2DConv输入：        (batch_size, C, H, W)
Linear输入：        仅对最后一个维度从输入变成输出
'''
import torch
import torch.nn as nn
from net.mytransformer import PositionalEncoding,TransformerWithPooling
from net.myswinunet import SwinTransformerSys
# import torch.nn.functional as F
from pytictoc import TicToc
from net.utils import get_model_memory_nolog
from net.jxtnet_pureTrans import TVL1Loss
def checksize(x):
    1
    print(x.shape, torch.prod(torch.tensor(x.shape)).item())

t = TicToc()
t.tic()

#------------------------------参数设置
cudadevice = 'cuda:0'
device = torch.device(cudadevice if torch.cuda.is_available() else "cpu")
tokenlength = 22500
hiddendim = 576
input_matrix = torch.randn(6, tokenlength, hiddendim).to(device)  # batchsize length dim
out_dim = 12
#------------------------------模型初始化
num_layers = 8
pool_size = 2  # 每次减少一半的序列长度

transformer_model = TransformerWithPooling(d_model=hiddendim, nhead=4, dim_feedforward=256, num_layers=num_layers, pool_size=pool_size, activation='silu').to(device)
get_model_memory_nolog(transformer_model)
pe = PositionalEncoding(d_model=hiddendim).to(device)
conv1d1 = nn.Conv1d(576, 1, kernel_size=1, stride=1, dilation=1 ,padding=0).to(device)
get_model_memory_nolog(conv1d1)
fc1d1 = nn.Sequential(
        nn.Linear(1406, 1406),
        nn.SiLU(),
        nn.Linear(1406, out_dim*8*45*90),
        nn.LayerNorm(out_dim*8*45*90)).to(device)
# fc1d1 = nn.Linear(351, 8*45*90)
get_model_memory_nolog(fc1d1)
swinunet = SwinTransformerSys(embed_dim=out_dim,window_size=9).to(device)
get_model_memory_nolog(swinunet)

#------------------------------数据流正文----------------------------
checksize(input_matrix)
# tic=toc(tic)
x = input_matrix.reshape(tokenlength, input_matrix.shape[0], -1)  # Transformer输入：Reshape to (seq_len, batch_size, input_channel) (L B C)
checksize(x)

#---------------Transformer Encoder----------------
print('进入Encoder')
x = pe(x)
x = transformer_model(x)  # 传入自定义的 Transformer 模型
# tic=toc(tic)

#---------------conv1d+fc bottleneck---------------
print("进入bottleneck")
x = x.reshape(x.shape[1], x.shape[2], -1)  # 1DConv输入：Reshape to (batch_size, input_channel, seq_len)
checksize(x)
x = conv1d1(x)
checksize(x)
x = fc1d1(x)
checksize(x)
# tic=toc(tic)

#-------------SwinTransformer Decoder--------------
print("进入Decoder")
x = x.reshape(x.shape[0],45*90,-1)
checksize(x)
x = swinunet(x)
checksize(x)
t.toc('准备及推理时间',restart=True)

l1loss = TVL1Loss(beta=1.0)
gt = torch.randn(6, 1, 360, 720).to(device)
loss = l1loss(x,gt).mean()
# print(loss)
loss.backward()
t.toc('反传时间',restart=True)




模型占用0.0243GB
模型占用0.0000GB
模型占用2.0482GB
SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:1000
---final upsample expand_first---
模型占用0.0004GB
torch.Size([6, 22500, 576]) 77760000
torch.Size([22500, 6, 576]) 77760000
进入Encoder
torch.Size([11250, 6, 576]) 38880000
torch.Size([5625, 6, 576]) 19440000
torch.Size([2812, 6, 576]) 9718272
torch.Size([1406, 6, 576]) 4859136
进入bottleneck
torch.Size([6, 576, 1406]) 4859136
torch.Size([6, 1, 1406]) 8436
torch.Size([6, 1, 388800]) 2332800
进入Decoder
torch.Size([6, 4050, 96]) 2332800
torch.Size([6, 16200, 48]) 4665600
torch.Size([6, 64800, 24]) 9331200
torch.Size([6, 259200, 12]) 18662400
torch.Size([6, 259200, 12]) 18662400
torch.Size([6, 1, 360, 720]) 1555200
准备及推理时间 5.630191 seconds.
反传时间 1.879369 seconds.


原始6层encoder

In [ ]:
'''
Transformer输入：   (seq_len, batch_size, C)
1DConv输入：        (batch_size, C, seq_len)
2DConv输入：        (batch_size, C, H, W)
Linear输入：        仅对最后一个维度从输入变成输出
'''
import torch
import torch.nn as nn
from net.mytransformer import PositionalEncoding,TransformerWithPooling
from net.myswinunet import SwinTransformerSys
# import torch.nn.functional as F
from pytictoc import TicToc
from net.utils import get_model_memory_nolog
from net.jxtnet_pureTrans import TVL1Loss
def checksize(x):
    1
    print(x.shape, torch.prod(torch.tensor(x.shape)).item())
    
t = TicToc()
t.tic()

#------------------------------参数设置
cudadevice = 'cuda:0'
device = torch.device(cudadevice if torch.cuda.is_available() else "cpu")
tokenlength = 22500
hiddendim = 576
input_matrix = torch.randn(6, tokenlength, hiddendim).to(device)  # batchsize length dim
out_dim = 12
#------------------------------模型初始化
num_layers = 6
pool_size = 2  # 每次减少一半的序列长度

transformer_model = TransformerWithPooling(d_model=hiddendim, nhead=4, dim_feedforward=256, num_layers=num_layers, pool_size=pool_size, activation='silu').to(device)
get_model_memory_nolog(transformer_model)
pe = PositionalEncoding(d_model=hiddendim).to(device)
conv1d1 = nn.Conv1d(576, 1, kernel_size=1, stride=1, dilation=1 ,padding=0).to(device)
get_model_memory_nolog(conv1d1)
fc1d1 = nn.Sequential(
        nn.Linear(351, 351),
        nn.SiLU(),
        nn.Linear(351, out_dim*8*45*90),
        nn.LayerNorm(out_dim*8*45*90)).to(device)
# fc1d1 = nn.Linear(351, 8*45*90)
get_model_memory_nolog(fc1d1)
swinunet = SwinTransformerSys(embed_dim=out_dim,window_size=9).to(device)
get_model_memory_nolog(swinunet)

#------------------------------数据流正文----------------------------
checksize(input_matrix)
# tic=toc(tic)
x = input_matrix.reshape(tokenlength, input_matrix.shape[0], -1)  # Transformer输入：Reshape to (seq_len, batch_size, input_channel) (L B C)
checksize(x)

#---------------Transformer Encoder----------------
print('进入Encoder')
x = pe(x)
x = transformer_model(x)  # 传入自定义的 Transformer 模型
# tic=toc(tic)

#---------------conv1d+fc bottleneck---------------
print("进入bottleneck")
x = x.reshape(x.shape[1], x.shape[2], -1)  # 1DConv输入：Reshape to (batch_size, input_channel, seq_len)
checksize(x)
x = conv1d1(x)
checksize(x)
x = fc1d1(x)
checksize(x)
# tic=toc(tic)

#-------------SwinTransformer Decoder--------------
print("进入Decoder")
x = x.reshape(x.shape[0],45*90,-1)
checksize(x)
x = swinunet(x)
checksize(x)
t.toc('准备及推理时间',restart=True)

l1loss = TVL1Loss(beta=1.0)
gt = torch.randn(6, 1, 360, 720).to(device)
loss = l1loss(x,gt).mean()
# print(loss)
loss.backward()
t.toc('反传时间',restart=True)




模型占用0.0364GB
模型占用0.0000GB
模型占用0.5132GB
SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:1000
---final upsample expand_first---
模型占用0.0004GB
torch.Size([6, 22500, 576]) 77760000
torch.Size([22500, 6, 576]) 77760000
进入Encoder


/home/jiangxiaotian/anaconda3/envs/jxtnet/lib/python3.9/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([11250, 6, 576]) 38880000
torch.Size([5625, 6, 576]) 19440000
torch.Size([2812, 6, 576]) 9718272
torch.Size([1406, 6, 576]) 4859136
torch.Size([703, 6, 576]) 2429568
torch.Size([351, 6, 576]) 1213056
进入bottleneck
torch.Size([6, 576, 351]) 1213056
torch.Size([6, 1, 351]) 2106
torch.Size([6, 1, 388800]) 2332800
进入Decoder
torch.Size([6, 4050, 96]) 2332800
torch.Size([6, 16200, 48]) 4665600
torch.Size([6, 64800, 24]) 9331200
torch.Size([6, 259200, 12]) 18662400
torch.Size([6, 259200, 12]) 18662400
torch.Size([6, 1, 360, 720]) 1555200
准备及推理时间 4.055781 seconds.
反传时间 1.303007 seconds.


各种拆分之后，还是应该loss.backward那四秒，只不过闪转腾挪

In [ ]:
import torch
import torch.nn as nn
from net.utils import checksize
from net.mytransformer import PositionalEncoding,TransformerWithPooling
from net.myswinunet import SwinTransformerSys
# import torch.nn.functional as F
from pytictoc import TicToc
from net.utils import get_model_memory_nolog
from net.jxtnet_pureTrans import TVL1Loss,MeshEncoderDecoder
import os
import re
import torch.utils.data.dataloader as DataLoader
from net.utils import meshRCSDataset, find_matching_files, process_files

t = TicToc()
t.tic()

rcsdir = r'/home/jiangxiaotian/datasets/traintest' #T7920 Liang
in_ems = []
rcss = []
for file in os.listdir(rcsdir):
    if '.pt' in file:
        # print(file)
        plane, theta, phi, freq= re.search(r"([a-zA-Z0-9]{4})_theta(\d+)phi(\d+)f(\d.+).pt", file).groups()
        theta = int(theta)
        phi = int(phi)
        freq = float(freq)
        in_em = [plane,theta,phi,freq]
        rcs = torch.load(os.path.join(rcsdir,file))
        in_ems.append(in_em)
        rcss.append(rcs)
dataset = meshRCSDataset(in_ems, rcss)
dataloader = DataLoader.DataLoader(dataset, batch_size=6, shuffle=True, num_workers=0) #创建DataLoader迭代器

#------------------------------参数设置
cudadevice = 'cuda:0'
device = torch.device(cudadevice if torch.cuda.is_available() else "cpu")
tokenlength = 22500
hiddendim = 576
# input_matrix = torch.randn(6, tokenlength, hiddendim).to(device)  # batchsize length dim
decoder_outdim = 12
#------------------------------模型初始化
num_layers = 6
pool_size = 2  # 每次减少一半的序列长度

autoencoder = MeshEncoderDecoder(
    num_discrete_coors = 128,
    device= device,
    paddingsize = 22500,
    decoder_outdim=decoder_outdim #决定了decoder的size 12L 6M 3S
).to(device)
t.toc('准备时间',restart=True)

for in_em1,rcs1 in dataloader:
    print('-----------------------')
    t.toc('开始时间',restart=True)
    objlist , ptlist = find_matching_files(in_em1[0], "./planes")
    planesur_faces, planesur_verts, _, geoinfo = process_files(objlist, device) #为了解决多batch变量长度不一样的问题 在这一步就已经padding到等长了
    face = planesur_faces.clone().detach().to(device)
    vert = planesur_verts.clone().detach().to(device)
    # geo = geoinfo.clone().detach().to(device)
    # inem = in_em1.clone().detach().to(device)
    gt = rcs1.clone().detach().to(device)
    t.toc('刚要进网络时间',restart=True)
    loss, *_ = autoencoder( #这里使用网络，是进去跑了forward 
        vertices = vert,
        faces = face, #torch.Size([batchsize, 33564, 3])
        geoinfo = geoinfo, #[area, volume, scale]
        in_em = in_em1,#.to(device)
        GT = gt, #这里放真值
        device = device,
    )
    t.toc('推理时间',restart=True)

    # l1loss = TVL1Loss(beta=1.0)
    # gt = torch.randn(6, 1, 360, 720).to(device)
    # loss = l1loss(x,gt).mean()
    # print(loss)
    loss.backward()
    t.toc('反传时间',restart=True)




  初始化开始 27.228296 seconds.
SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:1000
---final upsample expand_first---
  初始化结束 1.454404 seconds.
准备时间 1.652453 seconds.
-----------------------
开始时间 0.001173 seconds.
刚要进网络时间 0.236386 seconds.
  刚进来 0.242952 seconds.
  encoder 0.010540 seconds.
  decoder 1.746488 seconds.
  后处理 0.130223 seconds.
推理时间 1.888701 seconds.
反传时间 0.041992 seconds.
-----------------------
开始时间 0.001471 seconds.
刚要进网络时间 4.136201 seconds.
  刚进来 4.181268 seconds.
  encoder 0.009690 seconds.
  decoder 1.754822 seconds.
  后处理 0.130295 seconds.
推理时间 1.895566 seconds.
反传时间 0.157857 seconds.
-----------------------
开始时间 0.002699 seconds.
刚要进网络时间 4.039380 seconds.
  刚进来 4.200757 seconds.
  encoder 0.010183 seconds.
  decoder 1.762697 seconds.
  后处理 0.129596 seconds.
推理时间 1.903291 seconds.
反传时间 0.157922 seconds.
-----------------------
开始时间 0.003042 seconds.
刚要进网络时间 4.052004 seconds.
  刚进来 4.214041 seconds.
  e

感觉也不是encoder的问题，就是反传的问题

In [ ]:
import torch
import torch.nn as nn
from net.utils import checksize
from net.mytransformer import PositionalEncoding,TransformerWithPooling
from net.myswinunet import SwinTransformerSys
# import torch.nn.functional as F
from pytictoc import TicToc
from net.utils import get_model_memory_nolog
from net.jxtnet_pureTrans import TVL1Loss,MeshEncoderDecoder
import os
import re
import torch.utils.data.dataloader as DataLoader
from net.utils import meshRCSDataset, find_matching_files, process_files

t = TicToc()
t.tic()

rcsdir = r'/home/jiangxiaotian/datasets/traintest' #T7920 Liang
in_ems = []
rcss = []
for file in os.listdir(rcsdir):
    if '.pt' in file:
        # print(file)
        plane, theta, phi, freq= re.search(r"([a-zA-Z0-9]{4})_theta(\d+)phi(\d+)f(\d.+).pt", file).groups()
        theta = int(theta)
        phi = int(phi)
        freq = float(freq)
        in_em = [plane,theta,phi,freq]
        rcs = torch.load(os.path.join(rcsdir,file))
        in_ems.append(in_em)
        rcss.append(rcs)
dataset = meshRCSDataset(in_ems, rcss)
dataloader = DataLoader.DataLoader(dataset, batch_size=6, shuffle=True, num_workers=0) #创建DataLoader迭代器

#------------------------------参数设置
cudadevice = 'cuda:0'
device = torch.device(cudadevice if torch.cuda.is_available() else "cpu")
tokenlength = 22500
hiddendim = 576
input_matrix = torch.randn(6, tokenlength, hiddendim).to(device)  # batchsize length dim
decoder_outdim = 12
#------------------------------模型初始化
num_layers = 6
pool_size = 2  # 每次减少一半的序列长度

autoencoder = MeshEncoderDecoder( #这里实例化，是进去跑了init
    num_discrete_coors = 128,
    device= device,
    paddingsize = 22500,
    decoder_outdim=decoder_outdim #决定了decoder的size 12L 6M 3S
).to(device)
t.toc('准备时间',restart=True)

for in_em1,rcs1 in dataloader:
    print('-----------------------')
    t.toc('开始时间',restart=True)
    objlist , ptlist = find_matching_files(in_em1[0], "./planes")
    planesur_faces, planesur_verts, _, geoinfo = process_files(objlist, device) #为了解决多batch变量长度不一样的问题 在这一步就已经padding到等长了
    loss, *_ = autoencoder( #这里使用网络，是进去跑了forward 
        vertices = planesur_verts,
        faces = planesur_faces, #torch.Size([batchsize, 33564, 3])
        geoinfo = geoinfo, #[area, volume, scale]
        in_em = in_em1,#.to(device)
        GT = rcs1.to(device), #这里放真值
        device = device,
    )
    t.toc('推理时间',restart=True)

    # l1loss = TVL1Loss(beta=1.0)
    # gt = torch.randn(6, 1, 360, 720).to(device)
    # loss = l1loss(x,gt).mean()
    # print(loss)
    loss.backward()
    t.toc('反传时间',restart=True)




  初始化开始 0.995110 seconds.
SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:1000
---final upsample expand_first---
  初始化结束 1.287830 seconds.
准备时间 2.284415 seconds.
-----------------------
开始时间 0.004077 seconds.


/home/jiangxiaotian/anaconda3/envs/jxtnet/lib/python3.9/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


  encoder 2.171665 seconds.
  decoder 0.604875 seconds.
  后处理 0.257600 seconds.
推理时间 3.024815 seconds.
反传时间 4.365017 seconds.
-----------------------
开始时间 0.006521 seconds.
  encoder 4.697009 seconds.
  decoder 1.744252 seconds.
  后处理 0.130561 seconds.
推理时间 2.200673 seconds.
反传时间 0.156219 seconds.
-----------------------
开始时间 0.004101 seconds.
  encoder 4.185405 seconds.
  decoder 1.747972 seconds.
  后处理 0.130603 seconds.
推理时间 5.903793 seconds.
反传时间 0.157812 seconds.
-----------------------
开始时间 0.007045 seconds.
  encoder 4.202237 seconds.
  decoder 1.755050 seconds.
  后处理 0.129718 seconds.
推理时间 5.922308 seconds.
反传时间 0.157092 seconds.


In [4]:
import torch
import torch.nn as nn
from net.utils import checksize
from net.mytransformer import PositionalEncoding,TransformerWithPooling
from net.myswinunet import SwinTransformerSys
# import torch.nn.functional as F
from pytictoc import TicToc
from net.utils import get_model_memory_nolog
from net.jxtnet_pureTrans import TVL1Loss,MeshEncoderDecoder
import os
import re
import torch.utils.data.dataloader as DataLoader
from net.utils import meshRCSDataset, find_matching_files, process_files

t = TicToc()
t.tic()

rcsdir = r'/home/jiangxiaotian/datasets/traintest' #T7920 Liang
in_ems = []
rcss = []
for file in os.listdir(rcsdir):
    if '.pt' in file:
        # print(file)
        plane, theta, phi, freq= re.search(r"([a-zA-Z0-9]{4})_theta(\d+)phi(\d+)f(\d.+).pt", file).groups()
        theta = int(theta)
        phi = int(phi)
        freq = float(freq)
        in_em = [plane,theta,phi,freq]
        rcs = torch.load(os.path.join(rcsdir,file))
        in_ems.append(in_em)
        rcss.append(rcs)
dataset = meshRCSDataset(in_ems, rcss)
dataloader = DataLoader.DataLoader(dataset, batch_size=6, shuffle=True, num_workers=0) #创建DataLoader迭代器

#------------------------------参数设置
cudadevice = 'cuda:0'
device = torch.device(cudadevice if torch.cuda.is_available() else "cpu")
tokenlength = 22500
hiddendim = 576
input_matrix = torch.randn(6, tokenlength, hiddendim).to(device)  # batchsize length dim
decoder_outdim = 12
#------------------------------模型初始化
num_layers = 6
pool_size = 2  # 每次减少一半的序列长度

autoencoder = MeshEncoderDecoder( #这里实例化，是进去跑了init
    num_discrete_coors = 128,
    device= device,
    paddingsize = 22500,
    decoder_outdim=decoder_outdim #决定了decoder的size 12L 6M 3S
).to(device)

for in_em1,rcs1 in dataloader:
        objlist , ptlist = find_matching_files(in_em1[0], "./planes")
        planesur_faces, planesur_verts, _, geoinfo = process_files(objlist, device) #为了解决多batch变量长度不一样的问题 在这一步就已经padding到等长了
        loss, *_ = autoencoder( #这里使用网络，是进去跑了forward 
            vertices = planesur_verts,
            faces = planesur_faces, #torch.Size([batchsize, 33564, 3])
            geoinfo = geoinfo, #[area, volume, scale]
            in_em = in_em1,#.to(device)
            GT = rcs1.to(device), #这里放真值
            device = device,
        )
t.toc('准备及推理时间',restart=True)

# l1loss = TVL1Loss(beta=1.0)
# gt = torch.randn(6, 1, 360, 720).to(device)
# loss = l1loss(x,gt).mean()
# print(loss)
loss.backward()
t.toc('反传时间',restart=True)




SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:1000
---final upsample expand_first---
torch.Size([5, 12996, 3]) 194940
torch.Size([5, 22500, 576]) 64800000
torch.Size([22500, 5, 576]) 64800000
torch.Size([11250, 5, 576]) 32400000
torch.Size([5625, 5, 576]) 16200000
torch.Size([2812, 5, 576]) 8098560
torch.Size([1406, 5, 576]) 4049280
torch.Size([703, 5, 576]) 2024640
torch.Size([351, 5, 576]) 1010880
torch.Size([5, 576, 351]) 1010880
torch.Size([5, 1, 351]) 1755
torch.Size([5, 1, 388800]) 1944000
torch.Size([5, 4050, 96]) 1944000
torch.Size([5, 16200, 48]) 3888000
torch.Size([5, 64800, 24]) 7776000
torch.Size([5, 259200, 12]) 15552000
torch.Size([5, 259200, 12]) 15552000
torch.Size([5, 1, 360, 720]) 1296000
准备及推理时间 4.102780 seconds.
反传时间 0.043535 seconds.


In [ ]:
'''
Transformer输入：   (seq_len, batch_size, C)
1DConv输入：        (batch_size, C, seq_len)
2DConv输入：        (batch_size, C, H, W)
Linear输入：        仅对最后一个维度从输入变成输出
'''
import torch
import torch.nn as nn
from net.utils import checksize,toc
from net.mytransformer import PositionalEncoding,TransformerWithPooling
from net.myswinunet import SwinTransformerSys
# import torch.nn.functional as F
from pytictoc import TicToc
from net.utils import get_model_memory_nolog
from net.jxtnet_pureTrans import TVL1Loss

t = TicToc()
t.tic()

#------------------------------参数设置
cudadevice = 'cuda:0'
device = torch.device(cudadevice if torch.cuda.is_available() else "cpu")
tokenlength = 22500
hiddendim = 576
input_matrix = torch.randn(1, tokenlength, hiddendim).to(device)  # batchsize length dim
out_dim = 24
#------------------------------模型初始化
num_layers = 6
pool_size = 2  # 每次减少一半的序列长度

transformer_model = TransformerWithPooling(d_model=hiddendim, nhead=4, dim_feedforward=256, num_layers=num_layers, pool_size=pool_size, activation='silu').to(device)
get_model_memory_nolog(transformer_model)
pe = PositionalEncoding(d_model=hiddendim).to(device)
conv1d1 = nn.Conv1d(576, 1, kernel_size=1, stride=1, dilation=1 ,padding=0).to(device)
get_model_memory_nolog(conv1d1)
fc1d1 = nn.Sequential(
        nn.Linear(351, 351),
        nn.SiLU(),
        nn.Linear(351, out_dim*8*45*90),
        nn.LayerNorm(out_dim*8*45*90)).to(device)
# fc1d1 = nn.Linear(351, 8*45*90)
get_model_memory_nolog(fc1d1)
swinunet = SwinTransformerSys(embed_dim=out_dim,window_size=9).to(device)
get_model_memory_nolog(swinunet)

#------------------------------数据流正文----------------------------
checksize(input_matrix)
# tic=toc(tic)
x = input_matrix.reshape(tokenlength, input_matrix.shape[0], -1)  # Transformer输入：Reshape to (seq_len, batch_size, input_channel) (L B C)
checksize(x)

#---------------Transformer Encoder----------------
print('进入Encoder')
x = pe(x)
x = transformer_model(x)  # 传入自定义的 Transformer 模型
# tic=toc(tic)

#---------------conv1d+fc bottleneck---------------
print("进入bottleneck")
x = x.reshape(x.shape[1], x.shape[2], -1)  # 1DConv输入：Reshape to (batch_size, input_channel, seq_len)
checksize(x)
x = conv1d1(x)
checksize(x)
x = fc1d1(x)
checksize(x)
# tic=toc(tic)

#-------------SwinTransformer Decoder--------------
print("进入Decoder")
x = x.reshape(x.shape[0],45*90,-1)
checksize(x)
x = swinunet(x)
checksize(x)
t.toc('准备及推理时间',restart=True)

l1loss = TVL1Loss(beta=1.0)
gt = torch.randn(1, 1, 360, 720).to(device)
loss = l1loss(x,gt)
loss.backward()
t.toc('反传时间',restart=True)




模型占用0.0364GB
模型占用0.0000GB
模型占用1.0259GB
SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:1000
---final upsample expand_first---
模型占用0.0016GB
torch.Size([1, 22500, 576]) 12960000
torch.Size([22500, 1, 576]) 12960000
进入Encoder
torch.Size([11250, 1, 576]) 6480000
torch.Size([5625, 1, 576]) 3240000
torch.Size([2812, 1, 576]) 1619712
torch.Size([1406, 1, 576]) 809856
torch.Size([703, 1, 576]) 404928
torch.Size([351, 1, 576]) 202176
进入bottleneck
torch.Size([1, 576, 351]) 202176
torch.Size([1, 1, 351]) 351
torch.Size([1, 1, 777600]) 777600
进入Decoder
torch.Size([1, 4050, 192]) 777600
torch.Size([1, 16200, 96]) 1555200
torch.Size([1, 64800, 48]) 3110400
torch.Size([1, 259200, 24]) 6220800
torch.Size([1, 259200, 24]) 6220800
torch.Size([1, 1, 360, 720]) 259200
准备及推理时间 2.839354 seconds.
反传时间 1.520759 seconds.
